## Множественное наследование

В языке Python допускается множественное наследование, когда дочерний класс наследуется от нескольких родительских (базовых). Множественное наследование применяется не так часто, как обычное, но некоторые подходы к программированию его активно используют. Например, идея миксинов в Python реализуется как раз через множественное наследование.

Предположим, что каждый товар в интернет магазине описывается классом, который наследуется от базового класса Goods:

In [1]:
class Goods:
    def __init__(self, name, weight, price):
        print('init MixinLog')
        self.name = name
        self.weight = weight
        self.price = price
        
    def print_info(self):
        print(f'{self.name}, {self.weight}, {self.price}')

class NoteBook(Goods):
    pass

In [2]:
n = NoteBook('Asus', 1.7, 2199)

init MixinLog


In [3]:
n.print_info()

Asus, 1.7, 2199


Необходимо добавить функционал логирования товаров. Это можно сделать с помощью отдельного класса:

In [4]:
class MixinLog:
    ID = 0
    
    def __init__(self):
        print('init MixinLog')
        MixinLog.ID += 1
        self.id = MixinLog.ID
    
    def save_sell_log(self):
        print(f'Товар с id= {self.id} был продан')

Данный класс работает совершенно независимо от других классов и лишь добавляет функционал по логированию товаров с использованием их id. Такие классы называются миксины (примеси).

Его можно поместить в цепочку наследований класса NoteBook. Теперь получается, что класс NoteBook наследуется от двух классов: Goods и MixinLog:

In [5]:
class NoteBook(Goods, MixinLog):
    pass

И функционал этого класса содержит функционал обоих классов.

Для того, чтобы сработал инициализатор класса MixinLog, необходимо вызвать его в базовом классе через объект посредник super:

In [6]:
class Goods:
    def __init__(self, name, weight, price):
        print('init Goods')
        super().__init__()
        self.name = name
        self.weight = weight
        self.price = price
        
    def print_info(self):
        print(f'{self.name}, {self.weight}, {self.price}')
        
class MixinLog:
    ID = 0
    
    def __init__(self):
        print('init MixinLog')
        MixinLog.ID += 1
        self.id = MixinLog.ID
    
    def save_sell_log(self):
        print(f'Товар с id = {self.id} был продан')

class NoteBook(Goods, MixinLog):
    pass

In [7]:
n1 = NoteBook('Asus', 1.7, 2199)
n2 = NoteBook('Asus', 1.7, 2199)
n3 = NoteBook('Asus', 1.7, 2199)

init Goods
init MixinLog
init Goods
init MixinLog
init Goods
init MixinLog


In [8]:
n1.save_sell_log()

Товар с id = 1 был продан


При создании объекта класса NoteBook инициализатор сначала ищется в самом классе, а если его нет - то в первом базовом. Если его не оказалось и там - то во втором и так далее. Из-за того, что инициализатор есть в первом базовом классе, до второго (которым и является класс MixinLog) поиск не доходит. Именно поэтому необходимо вызывать его в инициализаторе вручную.

Но почему вызывается метод из второго базового класса, а не из класса, от которого наследуется класс Goods - object?

В Python существует специальный алгоритм обхода базовых классов при множественном наследовании. Сокращенно он называется MRO.  MRO - Method Resolution Order. В данном случае алгоритм MRO будет обходить базовые классы в следующем порядке: NoteBook => Goods => MixinLog => object. Эту цепочку можно посмотреть для любого класса следующим образом:

In [9]:
NoteBook.__mro__

(__main__.NoteBook, __main__.Goods, __main__.MixinLog, object)

Для того, чтобы избежать ошибок, в инициализаторах дополнительных базовых классов принято не использовать никаких параметров помимо self.

Если в классах прописаны методы с одинаковыми именами, то будет вызван тот метод, который находится ближе по иерархии наслевадования к тому классу, от объекта которого он был вызыван:

In [10]:
class Goods:
    def __init__(self, name, weight, price):
        print('init Goods')
        super().__init__()
        self.name = name
        self.weight = weight
        self.price = price
        
    def print_info(self):
        print(f'{self.name}, {self.weight}, {self.price}')
        
class MixinLog:
    ID = 0
    
    def __init__(self):
        print('init MixinLog')
        MixinLog.ID += 1
        self.id = MixinLog.ID
    
    def save_sell_log(self):
        print(f'Товар с id = {self.id} был продан')
    
    def print_info(self):
        print('метод print_info класса MixinLog')

class NoteBook(Goods, MixinLog):
    pass

In [11]:
n = NoteBook('Asus', 1.7, 2199)

init Goods
init MixinLog


In [12]:
n.print_info()

Asus, 1.7, 2199


Изменить это можно двумя способами. Или явно вызвать метод у класса MixinLog и передать ему объект:

In [13]:
MixinLog.print_info(n)

метод print_info класса MixinLog


Либо же переопределить метод в классе NoteBook, и внутри этого метода вызывать метод класса MixinLog:

In [14]:
class NoteBook(Goods, MixinLog):
    def print_info(self):
        MixinLog.print_info(self)

In [15]:
n = NoteBook('Asus', 1.7, 2199)

init Goods
init MixinLog


In [16]:
n.print_info()

метод print_info класса MixinLog
